<h2>Seguimiento a la Consolidación de Información</h2>

In [ ]:
import os
import pandas as pd
from pathlib import Path
import datetime
import arcpy

<h4>Parametrización de variables</h4>

In [ ]:
ruta_base = r"D:\Insumos PCCasa"
ruta_salida_csv = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\Seguimiento_Consolidacion"
nombre_csv = "seguimiento_consolidacion.csv"
extensiones = ['.shp','.xls','.xlsx','.csv','.txt','.doc','.docx','.dwg','.dxf','.pdf', '.geojson', '.json', '.xtf', '.ili',
               '.SHP','.XLS','.XLSX','.CSV','.TXT','.DOC','.DOCX','.DWG','.DXF','.PDF', '.GEOJSON', '.JSON', '.XTF', '.ILI']

<h4>Indexación de fuente diferentes a las bases de datos geográficas</h4>

In [ ]:
ruta_archivo = []
nomArchivo = []
extensionArchivo = []
pesoArchivoMbs = []
fechaCreacionArchivo = []
fuenteArchivo = []

for ruta, directorio, archivo in os.walk(ruta_base):
    for nombre_archivo in archivo:        
        ruta_archivos = os.path.join(ruta, nombre_archivo)
        rArchivos = Path(ruta_archivos)
        extensionArchivos = rArchivos.suffix
        for extBase in extensiones:
            if extBase == rArchivos.suffix:
                ruta_archivo.append(ruta_archivos)
                nomArchivo.append(rArchivos.stem)
                fuenteArchivo.append(ruta_archivos.split('\\')[2])                
                extensionArchivo.append(rArchivos.suffix)
                pesoArchivoMbs.append((os.path.getsize(rArchivos))/1000000)
                # ** DE UT a Formato Fecha
                marca_tiempo_creacion = os.path.getctime(rArchivos)
                fecha_creacion = datetime.datetime.fromtimestamp(marca_tiempo_creacion)
                fecha_formateada = fecha_creacion.strftime('%Y-%m-%d')
                fechaCreacionArchivo.append(fecha_formateada)
                print("Se documenta {0}".format(rArchivos.name))

<h4>Indexación de las bases de datos geográficas</h4>

In [ ]:
for ruta, directorios, archivo in os.walk(ruta_base):
    for nombre_directorio in directorios:        
        ruta_directorios = os.path.join(ruta, nombre_directorio)
        rDirectorios = Path(ruta_directorios)
        if rDirectorios.suffix == '.gdb':
                ruta_archivo.append(ruta_directorios)                
                nomArchivo.append(rDirectorios.stem)
                fuenteArchivo.append(ruta_directorios.split('\\')[2])
                extensionArchivo.append(rDirectorios.suffix)
                pesoArchivoMbs.append((os.path.getsize(rDirectorios))/1000000)
                # ** DE UT a Formato Fecha
                marca_tiempo_creacion = os.path.getctime(rDirectorios)
                fecha_creacion = datetime.datetime.fromtimestamp(marca_tiempo_creacion)
                fecha_formateada = fecha_creacion.strftime('%Y-%m-%d')
                fechaCreacionArchivo.append(fecha_formateada)
                print("Se documenta {0}".format(rDirectorios.name))


<h4>Preparación de Data Frame</h4>

In [ ]:

columnas = ['ruta_archivo','nomArchivo','extensionArchivo','pesoArchivoMbs','fechaCreacionArchivo','fuenteArchivo']

lista_archivos_recibidos = list(zip(ruta_archivo, nomArchivo, extensionArchivo, pesoArchivoMbs, fechaCreacionArchivo, fuenteArchivo))
df_lista_archivos_recibidos = pd.DataFrame(lista_archivos_recibidos, columns = columnas)
df_lista_archivos_recibidos.head(50)

df_lista_archivos_recibidos["tipoInsumo"] = None

<h4>Tipificación de insumos en dataframe</h4>

In [ ]:

# ** Documental
df_lista_archivos_recibidos.loc[(df_lista_archivos_recibidos['tipoInsumo'].isnull()) & 
                                ((df_lista_archivos_recibidos['extensionArchivo'] == '.pdf') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.PDF') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.doc') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.DOC') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.docx') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.DOCX')), 'tipoInsumo'] = 'Documental'

# ** Geográfica
df_lista_archivos_recibidos.loc[(df_lista_archivos_recibidos['tipoInsumo'].isnull()) & 
                                ((df_lista_archivos_recibidos['extensionArchivo'] == '.gdb') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.GDB') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.shp') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.SHP') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.geojson') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.GEOJSON')), 'tipoInsumo'] = 'Geografica'

# ** Alfanumérica
df_lista_archivos_recibidos.loc[(df_lista_archivos_recibidos['tipoInsumo'].isnull()) & 
                                ((df_lista_archivos_recibidos['extensionArchivo'] == '.xlsx') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.XLSX') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.xls') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.XLS') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.json') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.JSON') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.xtf') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.XTF') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.ili') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.ILI') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.txt') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.TXT') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.csv') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.CSV') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.dwg') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.DWG') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.dxf') |
                                 (df_lista_archivos_recibidos['extensionArchivo'] == '.DXF')), 'tipoInsumo'] = 'Geografica'

<h4>Generación de CSV</h4>

In [ ]:
df_lista_archivos_recibidos.to_csv(os.path.join(ruta_salida_csv,nombre_csv))